<a href="https://colab.research.google.com/github/jaesik103/paython_machin-learning/blob/main/8_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05-2 교차 검증과 그리드 서치
1. cross_validate : 섞는 전처리 필요
2. GridSearchCV : 교차검증 -> 하이퍼파라미터 자동으로 찾아줌 -> 최종모델훈련
3. RandomizedSearchCV : 연속된 하이퍼파라미터 값을 탐색할 때 유리. 랜덤샘플링을 통해 탐색량 조절. 샘플링으로 교차검증 -> 하이퍼파라미터 자동으로 찾아줌 -> 최종모델훈련 

In [2]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [4]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.996921300750433
0.8584615384615385


과대적합 발생

# K-fold cross validation
- default = 5
- 섞는 전처리를 해야함
- 회귀모델은 KFold 분할기사용, 분류모델은 StratifiedKFold 사용

In [6]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01080585, 0.01048946, 0.01233482, 0.01112819, 0.01031709]), 'score_time': array([0.00125527, 0.00122666, 0.00129485, 0.00125074, 0.00154376]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# GridSearchCV
- **Hyper-parameter** 탐색과, **교차 검증**을 한 번에 수행
- cv defualt=5

In [15]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [17]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [18]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


알아서 최적의 모델로 전체 훈련세트에 대해 학습함

In [19]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [20]:
print(gs.cv_results_['mean_test_score']) # 결과 값 저장소

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [21]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [23]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          } # 교차 검증 횟수 : 9 * 15 * 10 = 1350개

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [25]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [26]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# Random Search
- RandomizedSearchCV 사용. n_iter=샘플링횟수
- n_jobs = -1 : CPU모든 코어 사용

In [28]:
from scipy.stats import uniform, randint # uniform : 실숫값 , randint : 정숫값

In [29]:
rgen = randint(0, 10) # 0에서 10 사이의 범위를 갖는 객체 생성
rgen.rvs(10) # 객체에서 10개를 무작위로 추출

array([0, 5, 8, 8, 9, 2, 7, 8, 4, 3])

In [35]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 91, 106, 101,  73, 109, 100, 116,  98, 100, 106]))

In [37]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.28155604, 0.0375652 , 0.64107688, 0.0059436 , 0.88585732,
       0.45115764, 0.07066597, 0.57629439, 0.68224538, 0.18877618])

In [42]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), # 매개변수 값이 수치형이고, 연속적인 실숫값이라면 확률분포 객체 사용 가능
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1, 25)
          }

In [43]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5f92bbfc90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5f92d99510>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5f92d99f90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5f92d99250>},
                   random_state=42)

In [44]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [45]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [46]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
